Data Prep and Imports

In [92]:
import pandas as pd
import matplotlib.pyplot as plt

leak_data_raw =  pd.read_csv("Gasleak Data Sets/sensor_readings.csv")
leak_data_raw.columns = ["temp","Time","Sensor 1", "Sensor 2", "Sensor 3", "Sensor 4", "Sensor 5", "Sensor 6", "Sensor 7", "Sensor 8", "Sensor 9", "Sensor 10", "Sensor 11", "Sensor 12", "Sensor 13", "Sensor 14", "Sensor 15", "Sensor 16", "Sensor 17", "Sensor 18", "Sensor 19", "Sensor 20", "Sensor 21", "Sensor 22", "Sensor 23", "Sensor 24"]

weather_df = pd.read_csv("Gasleak Data Sets/weather_data.csv")
columns_to_average = ['Barometric_Pressure', 'Humidity', 'Temperature', 'Wind_Direction', 'Wind_Speed']
for col in columns_to_average:
    weather_df[col] = weather_df.groupby('timestamp')[col].transform('mean')
weather_df = weather_df.drop_duplicates()
weather_df['timestamp'] = pd.to_datetime(weather_df['timestamp']).astype(int) // 10**9
weather_df = weather_df.rename(columns={'timestamp': 'tTime'})

leak_rate_df = pd.read_csv("Gasleak Data Sets/leak_locations_and_rate.csv")
leak_rate_df =  leak_rate_df[['LeakRate', 'tStart', 'tEnd']]
leak_rate_df['LeakRate'] = leak_rate_df.groupby('tStart')['LeakRate'].transform('mean')
leak_rate_df = leak_rate_df.drop_duplicates()


/var/folders/qt/r87857tx74n48x9wj7f9wljw0000gn/T/ipykernel_17157/2588303935.py:12: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  weather_df['timestamp'] = pd.to_datetime(weather_df['timestamp']).astype(int) // 10**9


Merging Data Sets

In [93]:
# Leak and weather data merge
leak_data_raw['tTime'] = leak_data_raw['Time']
leak_data_raw['tTime'] = pd.to_datetime(leak_data_raw['tTime'], unit='s')
weather_df['tTime'] = pd.to_datetime(weather_df['tTime'], unit='s')

leak_data_raw['tTime'] = leak_data_raw['tTime'].dt.floor('min').dt.round('min')
df = pd.merge(leak_data_raw, weather_df, on='tTime', how='left')
df = df.drop(['temp', 'tTime'], axis=1)


In [96]:
idx = 0
df['Time'] = pd.to_datetime(df['Time'], unit='s')
leak_rate_df['tStart'] = pd.to_datetime(leak_rate_df['tStart'], unit='s')
leak_rate_df['tEnd'] = pd.to_datetime(leak_rate_df['tStart'], unit='s')
t = df.at[idx,'Time']

for index, row in leak_rate_df.iterrows():
    while t < row['tStart']:
        df.at[idx,'in_range'] = 0
        idx += 1
        t = df.at[idx,'Time']
    while t <= row['tEnd']:
        df.at[idx,'in_range'] = 1
        idx += 1
        t = df.at[idx,'Time']
for i in range(idx, len(df)):
    df.at[i,'in_range'] = 0
df

,Time,Sensor 1,Sensor 2,Sensor 3,Sensor 4,Sensor 5,Sensor 6,Sensor 7,Sensor 8,Sensor 9,...,Sensor 21,Sensor 22,Sensor 23,Sensor 24,Barometric_Pressure,Humidity,Temperature,Wind_Direction,Wind_Speed,in_range
0,2023-04-18 00:00:02,1030.0,1049.0,1166.0,964.0,1023.0,1090.5,1039.5,1153.0,1082.0,...,1040.0,1111.0,1080.0,1089.0,827.062090,31.740852,10.737401,320.568991,1.784585,0.0
1,2023-04-18 00:00:03,1033.0,1051.0,1163.0,974.0,993.0,1095.0,1044.0,1152.0,1085.0,...,1052.0,1105.5,1075.0,1095.0,827.062090,31.740852,10.737401,320.568991,1.784585,0.0
2,2023-04-18 00:00:04,1022.0,1051.0,1160.0,975.5,974.0,1081.0,1049.0,1133.0,1088.0,...,1033.0,1100.0,1021.0,1093.0,827.062090,31.740852,10.737401,320.568991,1.784585,0.0
3,2023-04-18 00:00:05,1034.0,1050.0,1165.0,977.0,988.0,1084.0,1019.0,1128.0,1091.0,...,1014.0,1107.0,1059.5,1110.0,827.062090,31.740852,10.737401,320.568991,1.784585,0.0
4,2023-04-18 00:00:06,1037.5,1049.0,1155.0,977.0,971.0,1084.0,1027.0,1144.0,1090.5,...,1019.0,1098.0,1098.0,1107.5,827.062090,31.740852,10.737401,320.568991,1.784585,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83408,2023-04-18 23:59:55,853.0,901.0,990.0,813.0,791.0,911.0,887.0,943.0,968.0,...,955.0,1020.0,949.0,987.0,822.476311,36.234596,11.388949,324.350733,8.866722,0.0
83409,2023-04-18 23:59:56,852.5,899.0,1007.0,801.0,788.5,900.0,890.0,931.0,984.0,...,954.0,1018.0,955.0,986.0,822.476311,36.234596,11.388949,324.350733,8.866722,0.0
83410,2023-04-18 23:59:57,852.0,901.0,1011.0,811.0,786.0,909.0,899.0,945.0,967.0,...,961.0,1017.0,938.0,982.0,822.476311,36.234596,11.388949,324.350733,8.866722,0.0
83411,2023-04-18 23:59:58,859.0,912.0,1010.0,811.0,764.0,909.0,892.0,959.0,969.0,...,963.0,1026.0,948.0,990.0,822.476311,36.234596,11.388949,324.350733,8.866722,0.0


In [97]:

# Convert time columns to datetime format
df['Time'] = pd.to_datetime(df['Time'], unit='s')
leak_rate_df['tStart'] = pd.to_datetime(leak_rate_df['tStart'], unit='s')
leak_rate_df['tEnd'] = pd.to_datetime(leak_rate_df['tEnd'], unit='s')

# Merge the DataFrames to create a cross-product
merged_df = pd.merge(df, leak_rate_df, how='cross')

# Create a new column 'in_range' based on the time range condition
merged_df['in_range'] = ((merged_df['Time'] >= merged_df['tStart']) &
                         (merged_df['Time'] < merged_df['tEnd'].shift(-1, fill_value=pd.Timestamp.max))).astype(int)

# Update the 'in_range' column in the original DataFrame
merged_df

,Time,Sensor 1,Sensor 2,Sensor 3,Sensor 4,Sensor 5,Sensor 6,Sensor 7,Sensor 8,Sensor 9,...,Sensor 24,Barometric_Pressure,Humidity,Temperature,Wind_Direction,Wind_Speed,in_range,LeakRate,tStart,tEnd
0,2023-04-18 00:00:02,1030.0,1049.0,1166.0,964.0,1023.0,1090.5,1039.5,1153.0,1082.0,...,1089.0,827.062090,31.740852,10.737401,320.568991,1.784585,0,50.629018,2023-04-18 00:00:35,2023-04-18 00:00:35
1,2023-04-18 00:00:02,1030.0,1049.0,1166.0,964.0,1023.0,1090.5,1039.5,1153.0,1082.0,...,1089.0,827.062090,31.740852,10.737401,320.568991,1.784585,0,45.247247,2023-04-18 00:15:35,2023-04-18 00:15:35
2,2023-04-18 00:00:02,1030.0,1049.0,1166.0,964.0,1023.0,1090.5,1039.5,1153.0,1082.0,...,1089.0,827.062090,31.740852,10.737401,320.568991,1.784585,0,45.247247,2023-04-18 00:15:35,2023-04-18 00:15:35
3,2023-04-18 00:00:02,1030.0,1049.0,1166.0,964.0,1023.0,1090.5,1039.5,1153.0,1082.0,...,1089.0,827.062090,31.740852,10.737401,320.568991,1.784585,0,50.629018,2023-04-18 00:45:35,2023-04-18 00:45:35
4,2023-04-18 00:00:02,1030.0,1049.0,1166.0,964.0,1023.0,1090.5,1039.5,1153.0,1082.0,...,1089.0,827.062090,31.740852,10.737401,320.568991,1.784585,0,45.247247,2023-04-18 01:00:35,2023-04-18 01:00:35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4921362,2023-04-18 23:59:59,855.0,900.0,1010.0,806.0,786.0,913.0,901.0,962.0,958.0,...,971.0,822.476311,36.234596,11.388949,324.350733,8.866722,0,54.017662,2023-04-18 21:16:58,2023-04-18 21:16:58
4921363,2023-04-18 23:59:59,855.0,900.0,1010.0,806.0,786.0,913.0,901.0,962.0,958.0,...,971.0,822.476311,36.234596,11.388949,324.350733,8.866722,0,0.000000,2023-04-18 21:19:58,2023-04-18 21:19:58
4921364,2023-04-18 23:59:59,855.0,900.0,1010.0,806.0,786.0,913.0,901.0,962.0,958.0,...,971.0,822.476311,36.234596,11.388949,324.350733,8.866722,0,85.664505,2023-04-18 21:20:58,2023-04-18 21:20:58
4921365,2023-04-18 23:59:59,855.0,900.0,1010.0,806.0,786.0,913.0,901.0,962.0,958.0,...,971.0,822.476311,36.234596,11.388949,324.350733,8.866722,0,0.000000,2023-04-18 21:23:20,2023-04-18 21:23:20
